This notebook contains the code used to wrangle, join, and analyze the following datasets:

+ American Community Survey (ACS) 5-year estimates (2015-2019)
    + [Aggregate](https://api.census.gov/data/2019/acs/acs5/variables.html)
    + [Profile](https://api.census.gov/data/2019/acs/acs5/profile/variables.html)

+ Chicago Community Area Census Tract Crosswalk
    + [Chicago Community Areas](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6)
    + [Chicago Census Tracts](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Census-Tracts-2010/5jrd-6zik)
+ [Chicago COVID-19 Community Vulnerability Index (CCVI)](https://data.cityofchicago.org/Health-Human-Services/Chicago-COVID-19-Community-Vulnerability-Index-CCV/xhc6-88s9)
+ [Hardship Index](https://data.cityofchicago.org/Health-Human-Services/hardship-index/792q-4jtu)



All csv files can be found in the "data" folder of the [working](https://github.com/danielgrzenda/broadbandequity/tree/working) branch of our Broadband Equity Github repo.

###### Importing Libraries

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

###### Importing Chicago ACS aggregate and profile data

In [4]:
# ACS aggregate

acs_agg = pd.read_csv("data/acs5_aggregate.csv",index_col=0,
                      parse_dates=[0]).drop(['state', 'county'], axis=1)

In [12]:
# ACS profile

acs_pro = pd.read_csv("data/acs5_profile.csv",
                      index_col=0,
                      parse_dates=[0]).drop(
                        ['estimated unemployment rate','state', 'county'], axis=1)

In [120]:
# merging both ACS datasets
# renaming variables

acs_df = acs_agg.merge(acs_pro, on='tract').rename(columns={
            'estimated total population': 'total_pop',
            'estimated total households': 'total_households',
            'estimated total with internet subscription': 'hh_internet',
            'estimated total with no internet access': 'hh_no_internet',
            'estimated total has a computer': 'hh_computer',
            'estimated total has no computer': 'hh_no_computer',
            'estimated total households with a computer': 'hh_computer',
            'estimated total households with broadband internet subscription': 'hh_broadband',
            'estimated total households median household income': 'hh_median_income',
            'percent estimated percent of families and people whose income in the past 12 months is below poverty level': 'p_poverty',
            'percent estimated unemployment rate': 'p_unemployed',
            'estimated total population hispanic or latino (of any race)': 'total_hispanic',
            'estimated total population Black or African American alone (non-Hispanic)': 'total_black'})


In [123]:
# 1319 rows x 14 columns
# variables beginning with "hh" indicate they are measured at the household level
# variables beginning with "p" indicate they are recorded as percentages 

acs_df.head(5)

,total_pop,hh_internet,hh_no_internet,hh_computer,hh_no_computer,tract,total_households,hh_computer,hh_broadband,hh_median_income,p_poverty,total_hispanic,total_black,p_unemployed
0,1825,392,149,426,149,630200,575,426,392,37422,25.7,1622,0,36.2
1,5908,1242,231,1411,133,580700,1544,1411,1242,47000,17.4,4742,161,42.3
2,3419,928,140,1068,104,590600,1172,1068,917,46033,7.9,2119,9,30.7
3,2835,917,138,1003,81,600700,1084,1003,917,45294,17.0,850,82,36.3
4,1639,322,245,356,218,611900,574,356,322,24507,55.0,438,1175,46.0


###### Importing Chicago COVID and Hardship Indices

In [124]:
# Chicago COVID hardship index csv

covid = pd.read_csv("data/covid_index.csv",
                        index_col=0,
                        parse_dates=[0]).reset_index()

In [125]:
# filtering for Community Areas only

covid = covid[covid['Geography Type']=="CA"]

In [126]:
# selecting and renaming relevant columns

covid = covid[['Community Area Name', 
               'Community Area or ZIP Code', 
               'CCVI Score', 
               'CCVI Category']].rename(columns={
                        "Community Area Name": "name",
                        'Community Area or ZIP Code': "comm_num",
                        "CCVI Score": "ccvi_score",
                        'CCVI Category': "ccvi_cat"})

In [127]:
# 77 rows x 4 columns 

covid.head(5)

,name,comm_num,ccvi_score,ccvi_cat
0,Ashburn,70,45.1,MEDIUM
2,Rogers Park,1,30.9,LOW
5,Lake View,6,5.2,LOW
7,Jefferson Park,11,25.6,LOW
8,Archer Heights,57,53.3,HIGH


In [128]:
# Chicago hardship index csv

hardship = pd.read_csv("data/hardship_index.csv",
                        index_col=0,
                        parse_dates=[0]).reset_index()

In [129]:
# removing "Chicago" community area

hardship = hardship[hardship['COMMUNITY AREA NAME']!="CHICAGO"]

In [130]:
# selecting and renaming relevant columns

hardship = hardship[['COMMUNITY AREA NAME', 
               'HARDSHIP INDEX']].rename(columns={
                        "COMMUNITY AREA NAME": "name",
                        'HARDSHIP INDEX': "hardship_score"})

In [131]:
# 77 rows x 2 columns

hardship.head(5)

,name,hardship_score
0,Rogers Park,39.0
1,West Ridge,46.0
2,Uptown,20.0
3,Lincoln Square,17.0
4,North Center,6.0


###### Importing Chicago Community Area Census Tract Crosswalk

In [132]:
# importing Census tracts mapped to community area number 
# renaming columns

tracts = pd.read_csv("data/tracts_comm_areas.csv",
                        index_col=0,
                        parse_dates=[0]).rename(columns={
                        "COMMAREA": "comm_num",          
                        "TRACTCE10": "tract"})

In [133]:
# importing community area numbers and names
# renaming columns

comm_area = pd.read_csv("data/comm_areas.csv",
                        index_col=0,
                        parse_dates=[0]).rename(columns={
                        "AREA_NUMBE": "comm_num"})

In [134]:
# merging both dataframes above to map tract wirh community area name

tract_area= comm_area.merge(tracts, on='comm_num')

In [135]:
# selecting columns we need and renaming them

tract_area=tract_area[['comm_num', 'tract']]

In [136]:
# 801 rows x 3 columns
# final dataframe

tract_area.head(5)

,comm_num,tract
0,35,842000
1,35,351500
2,35,839500
3,35,839200
4,35,839600


###### Joining ACS, COVID index, Hardship Index, and Community Areas

In [140]:
# merging ACS with chicago community areas, covid index, hardship index

full_df = tract_area.merge(acs_df, on='tract'
                           ).merge(covid, on='comm_num'
                                  ).merge(hardship, on='name')

In [138]:
# removing rows with no information

full_df = full_df[full_df['total_pop']!=0]

In [141]:
# 770 rows x 19 columns

full_df.head(5)

,comm_num,tract,total_pop,hh_internet,hh_no_internet,hh_computer,hh_no_computer,total_households,hh_computer,hh_broadband,hh_median_income,p_poverty,total_hispanic,total_black,p_unemployed,name,ccvi_score,ccvi_cat,hardship_score
0,35,842000,3164,340,16,347,15,362,347,340,99583,0.0,221,963,65.7,Douglas,36.0,MEDIUM,47.0
1,35,351500,731,281,56,314,30,344,314,281,16250,63.4,31,388,31.6,Douglas,36.0,MEDIUM,47.0
2,35,839500,1461,474,214,512,212,724,512,474,19950,0.0,53,992,53.9,Douglas,36.0,MEDIUM,47.0
3,35,839200,2405,1197,218,1246,179,1425,1246,1178,35707,11.5,98,1382,40.8,Douglas,36.0,MEDIUM,47.0
4,35,839600,1653,504,244,568,205,773,568,499,32401,6.2,122,1466,44.0,Douglas,36.0,MEDIUM,47.0


###### Computer, Internet, Broadband Access

This section will look into the community areas at a household-level to see who has and who does not have basic access to the internet and/or computer. We will also look into the households who have a broadband internet subscription. 

In [26]:
# who has a computer? who has internet access? 
# selecting columns we need

internet_df = full_df[['name', 
                        'total_households',
                        'no_internet', 
                        'has_internet', 
                        'has_computer', 
                        'no_computer',
                       'has_broadband']].groupby(by="name").sum()

In [27]:
# calculating who does not have a broadband internet subscription

internet_df['no_broadband']=internet_df['total_households']-internet_df['has_broadband']

In [28]:
# calculating percentages of households for each variable

# percentage of households with/out internet access

internet_df['p_no_internet']=internet_df['no_internet']/internet_df['total_households']*100
internet_df['p_has_internet']=internet_df['has_internet']/internet_df['total_households']*100

# percentage of households with/out no computer

internet_df['p_no_computer']=internet_df['no_computer']/internet_df['total_households']*100
internet_df['p_has_computer']=internet_df['has_computer']/internet_df['total_households']*100

# percentage of households with/out broadband 

internet_df['p_no_broadband']=internet_df['no_broadband']/internet_df['total_households']*100
internet_df['p_has_broadband']=internet_df['has_broadband']/internet_df['total_households']*100

In [63]:
# resulting dataframe

internet_df.sort_values(["p_has_broadband"],
                        ascending=True)

,total_households,no_internet,has_internet,has_computer,no_computer,has_broadband,no_broadband
name,,,,,,,
BURNSIDE,888,42.229730,53.716216,62.725225,37.274775,52.252252,47.747748
ENGLEWOOD,8983,34.598686,56.584660,74.897028,25.102972,56.306356,43.693644
WEST ENGLEWOOD,9483,38.089212,57.239270,70.336391,29.663609,57.239270,42.760730
FULLER PARK,1128,34.663121,59.042553,69.414894,30.585106,59.042553,40.957447
NORTH LAWNDALE,11075,29.986456,59.873589,76.000000,24.000000,59.620767,40.379233
...,...,...,...,...,...,...,...
LINCOLN SQUARE,18347,7.216439,89.960211,95.187224,4.812776,89.867553,10.132447
NORTH CENTER,14093,8.238132,90.562691,95.217484,4.782516,90.392393,9.607607
LINCOLN PARK,32395,6.254052,91.106652,94.786232,5.213768,91.029480,8.970520


Based on 2015-2019 ACS data, the percentages of households with computers, internet access, and internet broadband subscription are seen above. The neighborhoods of Burnside, Englewood, West Englewood, Fuller Park have the lowest percentages of both broadband subscription and internet access. The neighborhods of Near South Side, Lake View, Lincoln Park, and North Center have the highest. 

Broadband subscription and internet access numbers are extremely close, suggesting that the overwhelming majority of households who have access to internet do so via a broadbad subscription. 

###### Median Household Income & Poverty

In [64]:
# median household income and poverty rates by community area
# taking the median of median household incomes  

income_df = full_df[['name', 
                      'hh_median_income', 
                      'hh_percent_poverty']].groupby(by = "name").median().sort_values(["percent_poverty"], 
                                                                                  ascending = False)

income_df

,median_income,percent_poverty
name,,
RIVERDALE,15408.0,49.20
ENGLEWOOD,23125.0,36.60
WEST GARFIELD PARK,24001.5,36.40
EAST GARFIELD PARK,26500.5,33.05
GREATER GRAND CROSSING,29197.5,31.05
...,...,...
LOOP,105094.0,2.30
LINCOLN PARK,127177.5,1.85
NORTH CENTER,119904.5,1.80


Median income and poverty rates are based on household-level data. The medians of median household income and poverty rates across tracts were used to make the table above. The median household incomes in Chicago community areas range from Riverdale's \\$15,408 all the way to Lincoln Park's \\$127,177. Riverdale also has the highest percentage of their households in poverty at 49.2%. 

###### Race & Ethnicity

In [65]:
# race and ethnicity by community area

race_df = full_df[['name', 
                        'total_pop',
                        'total_hispanic', 
                        'total_black']].groupby(by = "name").sum().sort_values(["name"], 
                                                                      ascending = True)

In [66]:
# percentage of population hispanic

race_df['p_total_hispanic']=race_df['total_hispanic']/race_df['total_pop']*100

# percentage of population black non-hispanic

race_df['p_total_black']=race_df['total_black']/race_df['total_pop']*100

In [67]:
# final race and ethnicity dataframe

race_df.sort_values(["p_total_black"], ascending = False)

,total_pop,total_hispanic,total_black
name,,,
AVALON PARK,9713,0.092659,96.581901
BURNSIDE,2006,1.944167,96.261216
WASHINGTON HEIGHTS,26742,1.140528,96.096029
CHATHAM,30967,0.846062,95.666355
GREATER GRAND CROSSING,30149,1.738034,95.654914
...,...,...,...
FOREST GLEN,19384,15.131036,1.191704
JEFFERSON PARK,27503,24.746391,1.065338
ARCHER HEIGHTS,13726,77.415125,0.954393


Race and ethnicity data are based on total population numbers. Hispanic/Latino ethnicity was based on all races. Black/African-American race was non-Hispanic/Latino. Chicago community areas vary vastly in their race and ethnic compositions. Gage Park, South Lawndalw, West Elsdon, and Hermosa have the highest percentages of Hispanics/Latinos of all races. Calumet Heights, Washington Heights, Avalon Park, and Oakland have the highest percentages of non-Hispanic/Latino African-American/Blacks. 

##### Internet Access & Demographics Combined

The table below shows all of the variables above.

In [68]:
# merging all tables by community area

internet_demographics = internet_df.merge(income_df,
                        on='name').merge(race_df,
                                          on='name')

In [69]:
# reordering columns and showing final dataframe
internet_demographics = internet_demographics[['total_pop',
                      'total_households',
                      'p_has_computer',
                      'p_has_internet',
                      'p_has_broadband',
                      'hh_median_income',
                      'hh_percent_poverty',
                     'p_total_hispanic',
                     'p_total_black']].sort_values(["name"], 
                                                 ascending = True)


In [71]:
internet_demographics

,total_pop,total_households,has_computer,has_internet,has_broadband,median_income,percent_poverty,total_hispanic,total_black
name,,,,,,,,,
ALBANY PARK,49806,16909,89.325211,79.768171,79.703117,66818.0,12.10,44.972493,4.941172
ARCHER HEIGHTS,13726,3919,81.832100,73.641235,73.641235,48629.0,10.10,77.415125,0.954393
ARMOUR SQUARE,13538,5396,75.315048,68.291327,68.087472,33333.0,25.80,4.321170,8.383809
ASHBURN,43356,13124,90.269735,79.617495,79.564157,69261.0,8.75,41.327613,45.871390
AUBURN GRESHAM,45909,17161,79.972030,60.567566,59.967368,35568.0,23.60,2.180400,95.386526
...,...,...,...,...,...,...,...,...,...
WEST LAWN,31886,9272,84.156601,76.509922,75.711821,52992.5,12.55,83.971022,2.650066
WEST PULLMAN,30020,10598,84.657483,76.438951,76.203057,43143.0,15.50,5.203198,91.868754
WEST RIDGE,78466,25714,91.448238,81.181458,80.812009,53153.0,14.30,18.906278,11.579538


In [27]:
# export to Excel file in the data folder 

internet_demographics.to_csv("data/access_comm_area.csv") 

All variables except total_pop, total_households, and median_income are in percentages. All internet variables are on the household level. All demographic variables are on the community area population level. 

Based on 2015-2019 ACS data, the percentages of households with computers, internet access, and internet broadband subscription are seen above. The neighborhoods of Burnside, Englewood, West Englewood, Fuller Park have the lowest percentages of both broadband subscription and internet access. The neighborhods of Near South Side, Lake View, Lincoln Park, and North Center have the highest. 

Broadband subscription and internet access numbers are extremely close, suggesting that the overwhelming majority of households who have access to internet do so via a broadbad subscription. 

Median income and poverty rates are based on household-level data. The medians of median household income and poverty rates across tracts were used to make the table above. The median household incomes in Chicago community areas range from Riverdale's \\$15,408 all the way to Lincoln Park's \\$127,177. Riverdale also has the highest percentage of their households in poverty at 49.2%. 

Race and ethnicity data are based on total population numbers. Hispanic/Latino ethnicity was based on all races. Black/African-American race was non-Hispanic/Latino. Chicago community areas vary vastly in their race and ethnic compositions. Gage Park, South Lawndalw, West Elsdon, and Hermosa have the highest percentages of Hispanics/Latinos of all races. Calumet Heights, Washington Heights, Avalon Park, and Oakland have the highest percentages of non-Hispanic/Latino African-American/Blacks. 